In [1]:
from huggingface_hub import login
import inspect
import dotenv
import os

dotenv.load_dotenv()
hf_token = os.getenv("HUGGINGFACE_TOKEN")
login(hf_token)

/root/miniconda3/envs/gemma_xai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b-it",
    # device_map="auto",
    # torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
)
device = torch.device("cuda:0")
# model.to(device)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))


`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:29<00:00,  7.32s/it]


<bos>Write me a poem about Machine Learning.

In silicon valleys, where data flows,
A new intelligence, silently grows.
Machine Learning, a name whispered low,
Algorithms dance, where patterns


In [3]:
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(inputs["input_ids"], max_length=1000, num_return_sequences=1)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

SHAP, LIME 을 Gemma가 알고 있는지 체크

In [4]:
prompt = "What is SHAP and LIME? Plase explain with detail."

In [5]:
gemma_response = generate_response(prompt)
print(f"Gemma: {gemma_response}")

Gemma: What is SHAP and LIME? Plase explain with detail.

## SHAP and LIME: Unveiling the Black Box of Machine Learning

Machine learning models, especially deep learning ones, are often referred to as "black boxes." This means their internal workings are complex and difficult to interpret, making it challenging to understand how they arrive at their predictions. 

SHAP (SHapley Additive exPlanations) and LIME (Local Interpretable Model-agnostic Explanations) are two popular techniques used to shed light on these black boxes and provide interpretable explanations for model predictions.

**1. SHAP:**

* **Concept:** SHAP is based on the concept of game theory, specifically the Shapley values. In a game, each player contributes to the overall outcome. Shapley values assign a value to each player, representing their marginal contribution to the game's payoff.

* **Application:** In machine learning, SHAP values are used to quantify the contribution of each feature to a specific prediction

잘 알고 있다!

# 1. Zero-shot Prompting

Test natural language without format

In [6]:
prompt = inspect.cleandoc('''
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad”.
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please explain the reasons and relationships that led to the predictions, using the predicted probabilities and SHAP, LIME XAI analysis results in a specific and human-readable manner.

Prediction Probability:
Good - 0.57029474
Bad - 0.4297053
Predicted to Good

SHAP analysis (Feature, SHAP Importance):
(NumSatisfactoryTrades, 0.320243)
(PercentTradesNeverDelq, 0.305187)
(MSinceMostRecentInqexcl7days, 0.293064)
(NumTradesOpeninLast12M, 0.227222)
(MaxDelq2PublicRecLast12M, 0.212770)
(NumBank2NatlTradesWHighUtilization, 0.073882)
(NumTotalTrades, 0.065301)
(NumRevolvingTradesWBalance, 0.049232)
(MSinceMostRecentDelq, 0.042432)
(NumInqLast6M, 0.039852)

LIME analysis (Feature, LIME Importance):
(MSinceMostRecentInqexcl7days <= -7.00, -0.186905)
(6.00 < MaxDelq2PublicRecLast12M <= 7.00, -0.135623)
(96.00 < PercentTradesNeverDelq <= 100.00, -0.124091)
(NumSatisfactoryTrades > 27.00, -0.110831)
(NumTradesOpeninLast12M > 3.00,  0.058771)
(NumRevolvingTradesWBalance > 5.00, 0.036777)
(NumTotalTrades > 29.00, -0.027887)
(PercentInstallTrades <= 20.00, 0.027751)
(NumTrades90Ever2DerogPubRec <= 0.00, -0.019881)
(49.50 < PercentTradesWBalance <= 67.00, -0.013193)''')

In [7]:
gemma_response = generate_response(prompt)
print(f"Gemma: {gemma_response}")

Gemma: The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad”.
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please explain the reasons and relationships that led to the predictions, using the predicted probabilities and SHAP, LIME XAI analysis results in a specific and human-readable manner.

Prediction Probability:
Good - 0.57029474
Bad - 0.4297053
Predicted to Good

SHAP analysis (Feature, SHAP Importance):
(NumSatisfactoryTrades, 0.320243)
(PercentTradesNeverDelq, 0.305187)
(MSinceMostRecentInqexcl7days, 0.293064)
(NumTradesOpeninLast12M, 0.227222)
(MaxDelq2PublicRecLast12M, 0.212770)
(NumBank2NatlTradesWHighUtilization, 0.073882)
(NumTotalTrades, 0.065

Edited 1

In [8]:
prompt = inspect.cleandoc('''
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad”.
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please synthesize the key insights from the predicted probabilities, SHAP, and LIME XAI analysis results to explain the overall reasons and relationships that led to the predictions. Focus on providing a comprehensive conclusion that captures the most critical factors influencing the predictions over time, rather than delving into individual case analyses.
Please do not print the prompt you entered as it is.

Prediction Probability:
Good - 0.57029474
Bad - 0.4297053
Predicted to Good

SHAP analysis (Feature, SHAP Importance):
(NumSatisfactoryTrades, 0.320243)
(PercentTradesNeverDelq, 0.305187)
(MSinceMostRecentInqexcl7days, 0.293064)
(NumTradesOpeninLast12M, 0.227222)
(MaxDelq2PublicRecLast12M, 0.212770)
(NumBank2NatlTradesWHighUtilization, 0.073882)
(NumTotalTrades, 0.065301)
(NumRevolvingTradesWBalance, 0.049232)
(MSinceMostRecentDelq, 0.042432)
(NumInqLast6M, 0.039852)

LIME analysis (Feature, LIME Importance):
(MSinceMostRecentInqexcl7days <= -7.00, -0.186905)
(6.00 < MaxDelq2PublicRecLast12M <= 7.00, -0.135623)
(96.00 < PercentTradesNeverDelq <= 100.00, -0.124091)
(NumSatisfactoryTrades > 27.00, -0.110831)
(NumTradesOpeninLast12M > 3.00,  0.058771)
(NumRevolvingTradesWBalance > 5.00, 0.036777)
(NumTotalTrades > 29.00, -0.027887)
(PercentInstallTrades <= 20.00, 0.027751)
(NumTrades90Ever2DerogPubRec <= 0.00, -0.019881)
(49.50 < PercentTradesWBalance <= 67.00, -0.013193)''')

In [9]:
gemma_response = generate_response(prompt)
print(f"Gemma: {gemma_response}")

Gemma: The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad”.
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please synthesize the key insights from the predicted probabilities, SHAP, and LIME XAI analysis results to explain the overall reasons and relationships that led to the predictions. Focus on providing a comprehensive conclusion that captures the most critical factors influencing the predictions over time, rather than delving into individual case analyses.
Please do not print the prompt you entered as it is.

Prediction Probability:
Good - 0.57029474
Bad - 0.4297053
Predicted to Good

SHAP analysis (Feature, SHAP Importance):
(NumSatisfactoryTrades, 

Edited 2

In [10]:
prompt = inspect.cleandoc('''
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad”.
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please synthesize the key insights from the predicted probabilities, SHAP, and LIME XAI analysis results to explain the overall reasons and relationships that led to the predictions. Focus on providing a comprehensive conclusion that captures the most critical factors influencing the predictions over time, rather than delving into individual case analyses.
Please do not print the prompt you entered as it is.
Please response only conclusion part.

Prediction Probability:
Good - 0.57029474
Bad - 0.4297053
Predicted to Good

SHAP analysis (Feature, SHAP Importance):
(NumSatisfactoryTrades, 0.320243)
(PercentTradesNeverDelq, 0.305187)
(MSinceMostRecentInqexcl7days, 0.293064)
(NumTradesOpeninLast12M, 0.227222)
(MaxDelq2PublicRecLast12M, 0.212770)
(NumBank2NatlTradesWHighUtilization, 0.073882)
(NumTotalTrades, 0.065301)
(NumRevolvingTradesWBalance, 0.049232)
(MSinceMostRecentDelq, 0.042432)
(NumInqLast6M, 0.039852)

LIME analysis (Feature, LIME Importance):
(MSinceMostRecentInqexcl7days <= -7.00, -0.186905)
(6.00 < MaxDelq2PublicRecLast12M <= 7.00, -0.135623)
(96.00 < PercentTradesNeverDelq <= 100.00, -0.124091)
(NumSatisfactoryTrades > 27.00, -0.110831)
(NumTradesOpeninLast12M > 3.00,  0.058771)
(NumRevolvingTradesWBalance > 5.00, 0.036777)
(NumTotalTrades > 29.00, -0.027887)
(PercentInstallTrades <= 20.00, 0.027751)
(NumTrades90Ever2DerogPubRec <= 0.00, -0.019881)
(49.50 < PercentTradesWBalance <= 67.00, -0.013193)''')

In [11]:
gemma_response = generate_response(prompt)
print(f"Gemma: {gemma_response}")

Gemma: The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad”.
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please synthesize the key insights from the predicted probabilities, SHAP, and LIME XAI analysis results to explain the overall reasons and relationships that led to the predictions. Focus on providing a comprehensive conclusion that captures the most critical factors influencing the predictions over time, rather than delving into individual case analyses.
Please do not print the prompt you entered as it is.
Please response only conclusion part.

Prediction Probability:
Good - 0.57029474
Bad - 0.4297053
Predicted to Good

SHAP analysis (Feature, SHAP

Edited 3

In [12]:
prompt = inspect.cleandoc('''
Question:
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad”.
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please synthesize the key insights from the predicted probabilities, SHAP, and LIME XAI analysis results to explain the overall reasons and relationships that led to the predictions. Focus on providing a comprehensive conclusion that captures the most critical factors influencing the predictions over time, rather than delving into individual case analyses.

Context:
1. Prediction Probability
- Good: 0.57029474
- Bad: 0.4297053
- Predicted to Good

2. SHAP analysis (Feature, SHAP Importance)
- (NumSatisfactoryTrades, 0.320243)
- (PercentTradesNeverDelq, 0.305187)
- (MSinceMostRecentInqexcl7days, 0.293064)
- (NumTradesOpeninLast12M, 0.227222)
- (MaxDelq2PublicRecLast12M, 0.212770)
- (NumBank2NatlTradesWHighUtilization, 0.073882)
- (NumTotalTrades, 0.065301)
- (NumRevolvingTradesWBalance, 0.049232)
- (MSinceMostRecentDelq, 0.042432)
- (NumInqLast6M, 0.039852)

3. LIME analysis (Feature, LIME Importance)
- (MSinceMostRecentInqexcl7days <= -7.00, -0.186905)
- (6.00 < MaxDelq2PublicRecLast12M <= 7.00, -0.135623)
- (96.00 < PercentTradesNeverDelq <= 100.00, -0.124091)
- (NumSatisfactoryTrades > 27.00, -0.110831)
- (NumTradesOpeninLast12M > 3.00,  0.058771)
- (NumRevolvingTradesWBalance > 5.00, 0.036777)
- (NumTotalTrades > 29.00, -0.027887)
- (PercentInstallTrades <= 20.00, 0.027751)
- (NumTrades90Ever2DerogPubRec <= 0.00, -0.019881)
- (49.50 < PercentTradesWBalance <= 67.00, -0.013193)

Answer:''')

In [13]:
gemma_response = generate_response(prompt)
gemma_response = gemma_response.split('Answer:')[1]
print(f"Gemma: {gemma_response}")

Gemma: 

The XAI analysis reveals that the model predicts a "Good" RiskPerformance with a probability of 57%, indicating a moderate confidence level.  

**Key Factors Influencing the Prediction:**

* **Positive Credit History:** The model heavily emphasizes positive credit history indicators.  "NumSatisfactoryTrades" and "PercentTradesNeverDelq" emerge as the most influential features in SHAP analysis, suggesting a strong reliance on consistent on-time payments and a history of successful credit management.

* **Recent Credit Activity:** "MSinceMostRecentInqexcl7days" plays a significant role, indicating that the model considers the recency of credit inquiries. A longer time since the last inquiry might be interpreted as a lower risk.

* **Limited Recent Delinquencies:**  "MaxDelq2PublicRecLast12M" and "MSinceMostRecentDelq" are important features, suggesting that the model is sensitive to recent delinquency history. A lower maximum delinquency and a longer time since the last delinque

Edited4

In [14]:
prompt = inspect.cleandoc('''
Question:
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please synthesize the key insights from the predicted probabilities, SHAP, and LIME XAI analysis results to explain the overall reasons and relationships that led to the predictions. Focus on providing a comprehensive conclusion that captures the most critical factors influencing the predictions over time, rather than delving into individual case analyses.
Additionally, instead of focusing on individual feature analysis, provide a final, holistic conclusion based on the combined results. Please explain the findings in a way that avoids technical jargon, so that even non-experts in machine learning or finance can easily understand the explanation.

Context:
1. Prediction Probability
- Good: 0.57029474
- Bad: 0.4297053
- Predicted to Good

2. SHAP analysis (Feature, SHAP Importance)
- (NumSatisfactoryTrades, 0.320243)
- (PercentTradesNeverDelq, 0.305187)
- (MSinceMostRecentInqexcl7days, 0.293064)
- (NumTradesOpeninLast12M, 0.227222)
- (MaxDelq2PublicRecLast12M, 0.212770)
- (NumBank2NatlTradesWHighUtilization, 0.073882)
- (NumTotalTrades, 0.065301)
- (NumRevolvingTradesWBalance, 0.049232)
- (MSinceMostRecentDelq, 0.042432)
- (NumInqLast6M, 0.039852)

3. LIME analysis (Feature, LIME Importance)
- (MSinceMostRecentInqexcl7days <= -7.00, -0.186905)
- (6.00 < MaxDelq2PublicRecLast12M <= 7.00, -0.135623)
- (96.00 < PercentTradesNeverDelq <= 100.00, -0.124091)
- (NumSatisfactoryTrades > 27.00, -0.110831)
- (NumTradesOpeninLast12M > 3.00,  0.058771)
- (NumRevolvingTradesWBalance > 5.00, 0.036777)
- (NumTotalTrades > 29.00, -0.027887)
- (PercentInstallTrades <= 20.00, 0.027751)
- (NumTrades90Ever2DerogPubRec <= 0.00, -0.019881)
- (49.50 < PercentTradesWBalance <= 67.00, -0.013193)

Answer:''')

In [15]:
gemma_response = generate_response(prompt)
# gemma_response = gemma_response.split('Answer:')[1]
print(f"Gemma: {gemma_response}")

Gemma: Question:
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please synthesize the key insights from the predicted probabilities, SHAP, and LIME XAI analysis results to explain the overall reasons and relationships that led to the predictions. Focus on providing a comprehensive conclusion that captures the most critical factors influencing the predictions over time, rather than delving into individual case analyses.
Additionally, instead of focusing on individual feature analysis, provide a final, holistic conclusion based on the combined results. Please explain the findings in a way that avoids tech

Edited4 with max length increase

In [16]:
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(inputs["input_ids"], max_length=2000, num_return_sequences=1)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [17]:
prompt = inspect.cleandoc('''
Question:
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please synthesize the key insights from the predicted probabilities, SHAP, and LIME XAI analysis results to explain the overall reasons and relationships that led to the predictions. Focus on providing a comprehensive conclusion that captures the most critical factors influencing the predictions over time, rather than delving into individual case analyses.
Additionally, instead of focusing on individual feature analysis, provide a final, holistic conclusion based on the combined results. Please explain the findings in a way that avoids technical jargon, so that even non-experts in machine learning or finance can easily understand the explanation.

Context:
1. Prediction Probability
- Good: 0.57029474
- Bad: 0.4297053
- Predicted to Good

2. SHAP analysis (Feature, SHAP Importance)
- (NumSatisfactoryTrades, 0.320243)
- (PercentTradesNeverDelq, 0.305187)
- (MSinceMostRecentInqexcl7days, 0.293064)
- (NumTradesOpeninLast12M, 0.227222)
- (MaxDelq2PublicRecLast12M, 0.212770)
- (NumBank2NatlTradesWHighUtilization, 0.073882)
- (NumTotalTrades, 0.065301)
- (NumRevolvingTradesWBalance, 0.049232)
- (MSinceMostRecentDelq, 0.042432)
- (NumInqLast6M, 0.039852)

3. LIME analysis (Feature, LIME Importance)
- (MSinceMostRecentInqexcl7days <= -7.00, -0.186905)
- (6.00 < MaxDelq2PublicRecLast12M <= 7.00, -0.135623)
- (96.00 < PercentTradesNeverDelq <= 100.00, -0.124091)
- (NumSatisfactoryTrades > 27.00, -0.110831)
- (NumTradesOpeninLast12M > 3.00,  0.058771)
- (NumRevolvingTradesWBalance > 5.00, 0.036777)
- (NumTotalTrades > 29.00, -0.027887)
- (PercentInstallTrades <= 20.00, 0.027751)
- (NumTrades90Ever2DerogPubRec <= 0.00, -0.019881)
- (49.50 < PercentTradesWBalance <= 67.00, -0.013193)

Answer:''')

In [18]:
gemma_response = generate_response(prompt)
# gemma_response = gemma_response.split('Answer:')[1]
print(f"Gemma: {gemma_response}")

Gemma: Question:
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please synthesize the key insights from the predicted probabilities, SHAP, and LIME XAI analysis results to explain the overall reasons and relationships that led to the predictions. Focus on providing a comprehensive conclusion that captures the most critical factors influencing the predictions over time, rather than delving into individual case analyses.
Additionally, instead of focusing on individual feature analysis, provide a final, holistic conclusion based on the combined results. Please explain the findings in a way that avoids tech

더이상 output 이 잘리지 않는다

Edited 5 (CoT)

In [19]:
prompt = inspect.cleandoc('''
Question:
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.

Please follow these steps to explain the prediction:

1. Analyze the key features from the SHAP analysis, explaining how each feature contributes to the prediction.
2. Analyze the key features from the LIME analysis, explaining the contribution of each feature in terms of how it influences the prediction.
3. Based on the individual feature analyses from SHAP and LIME, synthesize the insights to provide a comprehensive conclusion. The conclusion should focus on how these features work together to influence the final prediction.
4. Instead of focusing on technical jargon, ensure that the final explanation is understandable to non-experts in machine learning or finance.

Context:
1. Prediction Probability
- Good: 0.57029474
- Bad: 0.4297053
- Predicted to Good

2. SHAP analysis (Feature, SHAP Importance)
- (NumSatisfactoryTrades, 0.320243)
- (PercentTradesNeverDelq, 0.305187)
- (MSinceMostRecentInqexcl7days, 0.293064)
- (NumTradesOpeninLast12M, 0.227222)
- (MaxDelq2PublicRecLast12M, 0.212770)
- (NumBank2NatlTradesWHighUtilization, 0.073882)
- (NumTotalTrades, 0.065301)
- (NumRevolvingTradesWBalance, 0.049232)
- (MSinceMostRecentDelq, 0.042432)
- (NumInqLast6M, 0.039852)

3. LIME analysis (Feature, LIME Importance)
- (MSinceMostRecentInqexcl7days <= -7.00, -0.186905)
- (6.00 < MaxDelq2PublicRecLast12M <= 7.00, -0.135623)
- (96.00 < PercentTradesNeverDelq <= 100.00, -0.124091)
- (NumSatisfactoryTrades > 27.00, -0.110831)
- (NumTradesOpeninLast12M > 3.00,  0.058771)
- (NumRevolvingTradesWBalance > 5.00, 0.036777)
- (NumTotalTrades > 29.00, -0.027887)
- (PercentInstallTrades <= 20.00, 0.027751)
- (NumTrades90Ever2DerogPubRec <= 0.00, -0.019881)
- (49.50 < PercentTradesWBalance <= 67.00, -0.013193)

Answer:
1. SHAP Analysis: First, explain each feature's SHAP importance and how it contributes to the final prediction.
2. LIME Analysis: Then, explain the individual feature importance from LIME and its role in the prediction.
3. Conclusion: Finally, synthesize the insights from both SHAP and LIME to provide a comprehensive, easy-to-understand conclusion.
''')

In [20]:
gemma_response = generate_response(prompt)
# gemma_response = gemma_response.split('Answer:')[1]
print(f"Gemma: {gemma_response}")

Gemma: Question:
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.

Please follow these steps to explain the prediction:

1. Analyze the key features from the SHAP analysis, explaining how each feature contributes to the prediction.
2. Analyze the key features from the LIME analysis, explaining the contribution of each feature in terms of how it influences the prediction.
3. Based on the individual feature analyses from SHAP and LIME, synthesize the insights to provide a comprehensive conclusion. The conclusion should focus on how these features work together to influence the final prediction.
4. Instead o

Edited 6 (One Shot from GPT)

In [21]:
one_shot_example = [
    {
        "input": inspect.cleandoc('''
        1. Prediction Probability
        - Good: 0.57029474
        - Bad: 0.4297053
        - Predicted to Good

        2. SHAP analysis (Feature, SHAP Importance)
        - (NumSatisfactoryTrades, 0.320243)
        - (PercentTradesNeverDelq, 0.305187)
        - (MSinceMostRecentInqexcl7days, 0.293064)
        - (NumTradesOpeninLast12M, 0.227222)
        - (MaxDelq2PublicRecLast12M, 0.212770)
        - (NumBank2NatlTradesWHighUtilization, 0.073882)
        - (NumTotalTrades, 0.065301)
        - (NumRevolvingTradesWBalance, 0.049232)
        - (MSinceMostRecentDelq, 0.042432)
        - (NumInqLast6M, 0.039852)

        3. LIME analysis (Feature, LIME Importance)
        - (MSinceMostRecentInqexcl7days <= -7.00, -0.186905)
        - (6.00 < MaxDelq2PublicRecLast12M <= 7.00, -0.135623)
        - (96.00 < PercentTradesNeverDelq <= 100.00, -0.124091)
        - (NumSatisfactoryTrades > 27.00, -0.110831)
        - (NumTradesOpeninLast12M > 3.00,  0.058771)
        - (NumRevolvingTradesWBalance > 5.00, 0.036777)
        - (NumTotalTrades > 29.00, -0.027887)
        - (PercentInstallTrades <= 20.00, 0.027751)
        - (NumTrades90Ever2DerogPubRec <= 0.00, -0.019881)
        - (49.50 < PercentTradesWBalance <= 67.00, -0.013193)
        '''),
        "output": inspect.cleandoc('''
        1. SHAP Analysis:

        SHAP (SHapley Additive exPlanations) helps to understand how much each feature contributes to the prediction (whether “Good” or “Bad”). Here’s a breakdown of the key features:

        - NumSatisfactoryTrades (0.320243): This is the most important feature. A higher number of satisfactory trades positively impacts the prediction, indicating that individuals with more positive credit history tend to be classified as “Good” (reliable in making payments).
        - PercentTradesNeverDelq (0.305187): A high percentage of trades that were never delinquent suggests strong payment behavior, pushing the prediction towards “Good.”
        - MSinceMostRecentInqexcl7days (0.293064): The time since the most recent inquiry, excluding the last 7 days, also plays a key role. A larger time gap generally reflects better credit management, supporting a “Good” prediction.
        - NumTradesOpeninLast12M (0.227222): The number of trades opened in the last year can signal recent borrowing behavior. Higher values here can imply more risk, but in this case, it contributes moderately to the “Good” prediction.
        - MaxDelq2PublicRecLast12M (0.212770): This refers to the maximum delinquency reported in the last year. A low value here suggests better credit health, aiding the “Good” classification.
        - NumBank2NatlTradesWHighUtilization (0.073882): This feature represents the number of bank trades with high credit utilization. Lower values are preferred, indicating better credit management.
        - NumTotalTrades (0.065301) and NumRevolvingTradesWBalance (0.049232): These represent the total number of trades and revolving trades with a balance. Higher values indicate more credit activity, but they play a smaller role in the prediction.
        - MSinceMostRecentDelq (0.042432) and NumInqLast6M (0.039852): These features indicate the time since the most recent delinquency and the number of recent inquiries, respectively. Smaller values help lean towards a “Good” prediction.

        2. LIME Analysis:

        LIME (Local Interpretable Model-Agnostic Explanations) breaks down the influence of features on this particular prediction. Here’s what the key features from LIME suggest:

        - MSinceMostRecentInqexcl7days <= -7.00 (-0.186905): This negatively affects the prediction. A shorter gap since the last inquiry can indicate recent financial stress, pushing towards a “Bad” prediction.
        - 6.00 < MaxDelq2PublicRecLast12M <= 7.00 (-0.135623): A higher delinquency in public records in the last 12 months strongly pushes the prediction toward “Bad.”
        - 96.00 < PercentTradesNeverDelq <= 100.00 (-0.124091): Even though high non-delinquent trades generally signal good credit health, LIME suggests that small nuances (near the 100% mark) can have a negative effect on the prediction, likely indicating excessive caution.
        - NumSatisfactoryTrades > 27.00 (-0.110831): A large number of satisfactory trades, while generally positive, might indicate that the borrower has an extensive credit history, which adds complexity to the risk assessment.
        - NumTradesOpeninLast12M > 3.00 (0.058771): Having more trades open in the last year slightly contributes to a “Good” prediction, perhaps indicating financial activity and engagement with credit, but it is a modest influence.
        - NumRevolvingTradesWBalance > 5.00 (0.036777): A higher number of revolving trades with a balance has a small positive impact, reflecting credit usage that is managed but not overwhelming.
        - NumTotalTrades > 29.00 (-0.027887): A large number of total trades negatively impacts the prediction, possibly suggesting overextension in credit usage.
        - PercentInstallTrades <= 20.00 (0.027751): A smaller portion of installment trades (compared to revolving trades) has a minor positive effect.
        - NumTrades90Ever2DerogPubRec <= 0.00 (-0.019881): Not having any derogatory public records has a small positive influence.
        - 49.50 < PercentTradesWBalance <= 67.00 (-0.013193): This range of trades with a balance slightly nudges the prediction toward “Bad,” possibly due to concerns about managing higher balances.

        3. Conclusion:

        Both SHAP and LIME analyses highlight the importance of a consumer’s credit behavior, particularly the number of satisfactory trades, delinquency history, and recent credit inquiries.

        - From the SHAP analysis, we see that having many satisfactory trades and a high percentage of non-delinquent trades contribute strongly to the “Good” classification. Features like the time since recent inquiries and past delinquencies also play a significant role in determining the prediction.
        - The LIME analysis offers additional nuance. It highlights that while some features like having many satisfactory trades and no delinquent records are generally good, specific thresholds (like the number of trades or recent inquiries) can have a negative effect. For example, a very recent inquiry or a slightly high delinquency rate can push the prediction towards “Bad.”

        Overall, the prediction of “Good” is driven by a mix of factors reflecting responsible credit behavior (many satisfactory trades, low delinquencies, and non-delinquent history). However, nuances like recent inquiries or higher-than-usual credit activity slightly temper this positive outlook. Together, these features balance out to result in a prediction of “Good” with moderate confidence.
        ''')
    }
    # Add more examples as needed for the FewShot approach
]

In [22]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# few_shot_prompt_template = '''
# Context:
# {input}

# Answer:
# {output}
# '''

# # Use PromptTemplate instead of raw string for the example prompt template
# few_shot_prompt_template = PromptTemplate(
#     input_variables=["input", "output"],
#     template="Context:\n{input}\n\nAnswer:\n{output}"
# )


# Use PromptTemplate instead of raw string for the example prompt
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Context:\n{input}\n\nAnswer:\n{output}"
)

# few_shot_prompt = FewShotPromptTemplate(
#     examples=one_shot_example,
#     example_prompt_template=few_shot_prompt_template,
#     prefix=inspect.cleandoc('''
#                             The following is a context about the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
#                             The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
#                             Please follow these steps to explain the prediction and offer a final conclusion:
#                             1. Analyze the key features from the SHAP analysis, explaining how each feature contributes to the prediction.
#                             2. Analyze the key features from the LIME analysis, explaining the contribution of each feature in terms of how it influences the prediction.
#                             3. Based on the individual feature analyses from SHAP and LIME, synthesize the insights to provide a comprehensive conclusion. The conclusion should focus on how these features work together to influence the final prediction.
#                             4. Instead of focusing on technical jargon, ensure that the final explanation is understandable to non-experts in machine learning or finance.
#                             '''),
#     suffix="Now, based on the above structure, analyze the following new context.\nContext:\n{input}\nAnswer:",
#     input_variables=["input"]
# )

# Construct FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=one_shot_example,
    example_prompt=example_prompt,  # Correct argument name
    prefix=inspect.cleandoc('''
        The following is a context about the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
        The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
        Please follow these steps to explain the prediction and offer a final conclusion:
        1. Analyze the key features from the SHAP analysis, explaining how each feature contributes to the prediction.
        2. Analyze the key features from the LIME analysis, explaining the contribution of each feature in terms of how it influences the prediction.
        3. Based on the individual feature analyses from SHAP and LIME, synthesize the insights to provide a comprehensive conclusion. The conclusion should focus on how these features work together to influence the final prediction.
        4. Instead of focusing on technical jargon, ensure that the final explanation is understandable to non-experts in machine learning or finance.
    '''),
    suffix="Now, based on the above structure, analyze the following new context.\nContext:\n{input}\nAnswer:",
    input_variables=["input"]
)

# Modify the generate_response function to use the FewShotPromptTemplate with CoT
def generate_response(prompt):
    # Use FewShotPromptTemplate to format the input with CoT reasoning
    formatted_prompt = few_shot_prompt.format(input=prompt)

    # Debug: print the formatted prompt for verification
    print(formatted_prompt)

    # Tokenize and generate the response
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
    
    # Debug: ensure input IDs are correct
    print(f"Tokenized input: {inputs['input_ids']}")
    
    output = model.generate(inputs["input_ids"], max_length=4000, num_return_sequences=1)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return response


input_context = inspect.cleandoc('''
    1. Prediction Probability
    - Good: 0.57029474
    - Bad: 0.4297053
    - Predicted to Good

    2. SHAP analysis (Feature, SHAP Importance)
    - (NumSatisfactoryTrades, 0.320243)
    - (PercentTradesNeverDelq, 0.305187)
    - (MSinceMostRecentInqexcl7days, 0.293064)
    - (NumTradesOpeninLast12M, 0.227222)
    - (MaxDelq2PublicRecLast12M, 0.212770)
    - (NumBank2NatlTradesWHighUtilization, 0.073882)
    - (NumTotalTrades, 0.065301)
    - (NumRevolvingTradesWBalance, 0.049232)
    - (MSinceMostRecentDelq, 0.042432)
    - (NumInqLast6M, 0.039852)

    3. LIME analysis (Feature, LIME Importance)
    - (MSinceMostRecentInqexcl7days <= -7.00, -0.186905)
    - (6.00 < MaxDelq2PublicRecLast12M <= 7.00, -0.135623)
    - (96.00 < PercentTradesNeverDelq <= 100.00, -0.124091)
    - (NumSatisfactoryTrades > 27.00, -0.110831)
    - (NumTradesOpeninLast12M > 3.00,  0.058771)
    - (NumRevolvingTradesWBalance > 5.00, 0.036777)
    - (NumTotalTrades > 29.00, -0.027887)
    - (PercentInstallTrades <= 20.00, 0.027751)
    - (NumTrades90Ever2DerogPubRec <= 0.00, -0.019881)
    - (49.50 < PercentTradesWBalance <= 67.00, -0.013193)
''')

In [23]:
gemma_response = generate_response(input_context)
# gemma_response = gemma_response.split('Answer:')[1]
print(f"Gemma: {gemma_response}")

The following is a context about the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please follow these steps to explain the prediction and offer a final conclusion:
1. Analyze the key features from the SHAP analysis, explaining how each feature contributes to the prediction.
2. Analyze the key features from the LIME analysis, explaining the contribution of each feature in terms of how it influences the prediction.
3. Based on the individual feature analyses from SHAP and LIME, synthesize the insights to provide a comprehensive conclusion. The conclusion should focus on how these features work together to influence the fina

with 2nd sample to input

In [24]:
input_context = inspect.cleandoc('''
    1. Prediction Probability
    - Good: 0.57029474
    - Bad: 0.4297053
    - Predicted to Good

    2. SHAP analysis (Feature, SHAP Importance)
    - (NumSatisfactoryTrades, 0.320243)
    - (PercentTradesNeverDelq, 0.305187)
    - (MSinceMostRecentInqexcl7days, 0.293064)
    - (NumTradesOpeninLast12M, 0.227222)
    - (MaxDelq2PublicRecLast12M, 0.212770)
    - (NumBank2NatlTradesWHighUtilization, 0.073882)
    - (NumRevolvingTradesWBalance, 0.049232)
    - (MSinceMostRecentDelq, 0.042432)
    - (NumInqLast6M, 0.039852)

    3. LIME analysis (Feature, LIME Importance)
    - (MSinceMostRecentInqexcl7days <= -7.00, -0.186905)
    - (6.00 < MaxDelq2PublicRecLast12M <= 7.00, -0.135623)
    - (96.00 < PercentTradesNeverDelq <= 100.00, -0.124091)
    - (NumSatisfactoryTrades > 27.00, -0.110831)
    - (NumTradesOpeninLast12M > 3.00, 0.058771)
    - (NumRevolvingTradesWBalance > 5.00, 0.036777)
    - (NumTotalTrades > 29.00, -0.027887)
    - (PercentInstallTrades <= 20.00, 0.027751)
    - (NumTrades90Ever2DerogPubRec <= 0.00, -0.019881)
    - (49.50 < PercentTradesWBalance <= 67.00, -0.013193)
''')

In [25]:
gemma_response = generate_response(input_context)
# gemma_response = gemma_response.split('Answer:')[1]
print(f"Gemma: {gemma_response}")

The following is a context about the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please follow these steps to explain the prediction and offer a final conclusion:
1. Analyze the key features from the SHAP analysis, explaining how each feature contributes to the prediction.
2. Analyze the key features from the LIME analysis, explaining the contribution of each feature in terms of how it influences the prediction.
3. Based on the individual feature analyses from SHAP and LIME, synthesize the insights to provide a comprehensive conclusion. The conclusion should focus on how these features work together to influence the fina